# Imports

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table, Column,Integer, String
from sqlalchemy import insert, select, update, delete

In [ ]:
import sqlalchemy
sqlalchemy.__version__  

'1.4.40'

# Variables globales

In [ ]:
Database_URL = "sqlite:///db.sqlite"

engine = create_engine(Database_URL)

metadata = MetaData()

# Metodo 

In [ ]:
def querys(query):
    with engine.connect() as conn:
        conn.execute(query)

# Tabla

In [ ]:
users = Table(
    "users",
    metadata,
    Column("id_user",Integer, primary_key=True, autoincrement=True),
    Column("nombre",String(50), nullable=False),
    Column("email",String(50), nullable=False)
)

metadata.create_all(engine)

# Ingresar

In [ ]:
stmt = insert(users).values(nombre="Yael",email="yael@gmail.com")

querys(stmt)

# Insert Varios Registros a la vez

In [ ]:
data = [
    {"nombre":"Cliente 1","email":"cliente1@gmail.com"},
    {"nombre":"Cliente 2","email":"cliente@gmail.com"}
    ]

stmt = insert(users).values(data)

querys(stmt)

# Select

In [ ]:
query = select(users)

with engine.connect() as conn:
    result = conn.execute(query)
    for row in result.fetchall():
        print(f"Name: {row.nombre}, email: {row.email}")

Name: user #5, email: cliente1@gmail.com
Name: Cliente 2, email: cliente@gmail.com


## Select con  filtros

## Select con and

In [ ]:
from sqlalchemy import and_

query = select(users).where((users.c.id_user == 2) & (users.c.email == "cliente1@gmail.com")) # Opcion 1

query = select(users).where(and_(users.c.email == "cliente1@gmail.com",users.c.id_user == 2)) # Opcion 2

with engine.connect() as conn:
    result = conn.execute(query)
    for row in result.fetchall():
        print(row)

(2, 'user #5', 'cliente1@gmail.com')


## Select con or

In [ ]:
from operator import or_

query = select(users).where(or_(users.c.email == "cliente1@gmail.com",users.c.nombre.like("C%")))

with engine.connect() as conn:
    result = conn.execute(query)
    for row in result.fetchall():
        print(row)

(2, 'user', 'cliente1@gmail.com')
(3, 'Cliente 2', 'cliente@gmail.com')


## Select con Like

---

* %C% que contenga alguna C
* _C% que C este en la segunda posicion
* %C que termine en C
* C% que empieze en C

In [ ]:
query = select(users).where((users.c.nombre.like("%2")))

with engine.connect() as conn:
    result = conn.execute(query)
    for row in result.fetchall():
        print(row)

(3, 'Cliente 2', 'cliente@gmail.com')


## Select con in_

In [ ]:
query = select(users).where((users.c.nombre.in_(["Cliente 2",users.c.nombre.like("%user%")])))

with engine.connect() as conn:
    result = conn.execute(query)
    for row in result.fetchall():
        print(row)

(3, 'Cliente 2', 'cliente@gmail.com')


## Select con not_in

In [ ]:
query = select(users).where((users.c.nombre.not_in(["Cliente 2"])))

with engine.connect() as conn:
    result = conn.execute(query)
    for row in result.fetchall():
        print(row)

(2, 'user', 'cliente1@gmail.com')


# Update

In [ ]:
query = update(users).where(users.c.id_user == 2).values(nombre='user')

querys(query)

# Delete

In [ ]:
query = delete(users).where(users.c.id_user == 1)

querys(query)